<a href="https://colab.research.google.com/github/Vedmani/wear-particle-classification/blob/main/EfficientNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install wandb -qU
!wandb login

     |████████████████████████████████| 1.9 MB 13.7 MB/s 
     |████████████████████████████████| 182 kB 61.6 MB/s 
     |████████████████████████████████| 166 kB 59.7 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
     |████████████████████████████████| 166 kB 45.9 MB/s 
     |████████████████████████████████| 162 kB 69.7 MB/s 
     |████████████████████████████████| 162 kB 81.6 MB/s 
     |████████████████████████████████| 158 kB 81.2 MB/s 
     |████████████████████████████████| 157 kB 81.2 MB/s 
     |████████████████████████████████| 157 kB 61.6 MB/s 
     |████████████████████████████████| 157 kB 71.2 MB/s 
     |████████████████████████████████| 157 kB 69.9 MB/s 
     |████████████████████████████████| 157 kB 69.8 MB/s 
     |████████████████████████████████| 157 kB 63.5 MB/s 
     |████████████████████████████████| 157 kB 64.6 MB/s 
     |████████████████████████████████| 156 kB 85.7 MB/s 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import wandb
from wandb.keras import WandbCallback

In [4]:
tf.keras.utils.set_random_seed(1)
tf.config.experimental.enable_op_determinism()

In [14]:
PATH = '/content/drive/MyDrive/Research Internship/final data/wear particles/Train'
test_path = '/content/drive/MyDrive/Research Internship/final data/wear particles/Test'
batch_size = 32
img_height = 224
img_width = 224

train_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="training",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="validation",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

test_ds = tf.keras.utils.image_dataset_from_directory(
  test_path,
  seed = 42,
  image_size=(img_height, img_width))

class_names = train_ds.class_names
test_class_names = test_ds.class_names
num_classes = len(class_names)
print(class_names)

Found 879 files belonging to 5 classes.
Using 704 files for training.
Found 879 files belonging to 5 classes.
Using 175 files for validation.
Found 94 files belonging to 5 classes.
['cutting wear', 'normal rubbing and cutting wear combined', 'normal rubbing wear', 'red oxide', 'severe sliding wear']


In [15]:
efficientnet = tf.keras.applications.EfficientNetB0(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling='avg',
    classes=1000,
    classifier_activation="softmax"
    )

model_efficientnet=Sequential([
  #layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  efficientnet,
  #layers.Flatten(),
  #layers.Dense(32, activation='relu'),
  layers.Dropout(0.2), 
  layers.Dense(num_classes, activation='softmax')
])
efficientnet.trainable=False
model_efficientnet.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb0 (Functional)  (None, 1280)             4049571   
                                                                 
 dropout_3 (Dropout)         (None, 1280)              0         
                                                                 
 dense_3 (Dense)             (None, 5)                 6405      
                                                                 
Total params: 4,055,976
Trainable params: 6,405
Non-trainable params: 4,049,571
_________________________________________________________________


In [16]:
model_efficientnet.compile(
              optimizer= tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name='Adam'),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy']
              )

In [5]:
wandb.init(
      # Set the project where this run will be logged
      project="Transfer_Learning_New", 
      name = "EfficientnetB0"
)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: vedmanivaidya. Use `wandb login --relogin` to force relogin


In [ ]:
%%time
epochs=40
history = model_efficientnet.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
callbacks=[WandbCallback()]
)

Epoch 1/40
22/22 [==============================] - ETA: 0s - loss: 1.0463 - accuracy: 0.5952

wandb: Adding directory to artifact (/content/wandb/run-20221106_125215-usyie0td/files/model-best)... Done. 0.2s


22/22 [==============================] - 214s 7s/step - loss: 1.0463 - accuracy: 0.5952 - val_loss: 0.6411 - val_accuracy: 0.7886
Epoch 2/40
22/22 [==============================] - ETA: 0s - loss: 0.4826 - accuracy: 0.8537

wandb: Adding directory to artifact (/content/wandb/run-20221106_125215-usyie0td/files/model-best)... Done. 0.2s


22/22 [==============================] - 39s 2s/step - loss: 0.4826 - accuracy: 0.8537 - val_loss: 0.4087 - val_accuracy: 0.8743
Epoch 3/40
22/22 [==============================] - ETA: 0s - loss: 0.3196 - accuracy: 0.9361

...

22/22 [==============================] - 44s 2s/step - loss: 0.0351 - accuracy: 0.9957 - val_loss: 0.0770 - val_accuracy: 0.9829
Epoch 31/40
22/22 [==============================] - 3s 120ms/step - loss: 0.0307 - accuracy: 0.9972 - val_loss: 0.0776 - val_accuracy: 0.9771
Epoch 32/40
22/22 [==============================] - ETA: 0s - loss: 0.0289 - accuracy: 0.9972

wandb: Adding directory to artifact (/content/wandb/run-20221106_125215-usyie0td/files/model-best)... Done. 0.2s


22/22 [==============================] - 39s 2s/step - loss: 0.0289 - accuracy: 0.9972 - val_loss: 0.0765 - val_accuracy: 0.9829
Epoch 33/40
22/22 [==============================] - ETA: 0s - loss: 0.0304 - accuracy: 0.9972

wandb: Adding directory to artifact (/content/wandb/run-20221106_125215-usyie0td/files/model-best)... Done. 0.2s


22/22 [==============================] - 45s 2s/step - loss: 0.0304 - accuracy: 0.9972 - val_loss: 0.0763 - val_accuracy: 0.9771
Epoch 34/40
10/22 [============>.................] - ETA: 1s - loss: 0.0293 - accuracy: 0.9969

In [6]:
wandb.finish()

In [7]:
!nvidia-smi --query-gpu=gpu_name --format=csv

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.

